# Practical Machine Learning Project

## Background

Using devices such as Jawbone Up, Nike FuelBand, and Fitbit it is now possible to collect a large amount of data about personal activity relatively inexpensively. These type of devices are part of the quantified self movement – a group of enthusiasts who take measurements about themselves regularly to improve their health, to find patterns in their behavior, or because they are tech geeks. One thing that people regularly do is quantify how much of a particular activity they do, but they rarely quantify how well they do it. In this project, your goal will be to use data from accelerometers on the belt, forearm, arm, and dumbell of 6 participants. They were asked to perform barbell lifts correctly and incorrectly in 5 different ways. More information is available from the website here: http://groupware.les.inf.puc-rio.br/har (see the section on the Weight Lifting Exercise Dataset).

## Data

The training data for this project are available here:

https://d396qusza40orc.cloudfront.net/predmachlearn/pml-training.csv

The test data are available here:

https://d396qusza40orc.cloudfront.net/predmachlearn/pml-testing.csv


The data for this project come from this source: http://groupware.les.inf.puc-rio.br/har. If you use the document you create for this class for any purpose please cite them as they have been very generous in allowing their data to be used for this kind of assignment.

## Installing Essential Libraries

In [ ]:
#install.packages("ElemStatLearn",dependencies= TRUE, "/Users/Aquilesja/anaconda/lib/R/library",repos='http://cran.us.r-project.org')
#install.packages("AppliedPredictiveModeling",dependencies= TRUE, "/Users/Aquilesja/anaconda/lib/R/library",repos='http://cran.us.r-project.org')
#install.packages("elasticnet",dependencies= TRUE, "/Users/Aquilesja/anaconda/lib/R/library",repos='http://cran.us.r-project.org')
#install.packages("lubridate",dependencies= TRUE, "/Users/Aquilesja/anaconda/lib/R/library",repos='http://cran.us.r-project.org')
#install.packages("forecast",dependencies= TRUE, "/Users/Aquilesja/anaconda/lib/R/library",repos='http://cran.us.r-project.org')
#install.packages("RGtk2",dependencies= TRUE, "/Users/Aquilesja/anaconda/lib/R/library",repos='http://cran.us.r-project.org')
#install.packages("rattle", dependencies= TRUE,"/Users/Aquilesja/anaconda/lib/R/library",repos='http://cran.us.r-project.org')

## Importing needed Libraries

In [3]:
library(caret)
library(rpart)
library(rpart.plot)
library(randomForest)
library(corrplot)
library(ElemStatLearn)


## Reading traininf and testind data from CSV files

In [4]:
trainData <- read.csv("pml-training.csv")
testData <- read.csv("pml-testing.csv")
cat("The training set contains ", dim(trainData)[1], "Individuals", " and ",dim(trainData)[2], "Variables\n")
cat("The testing set contains ", dim(testData)[1],"Individuals", " and ",dim(testData)[2], "Variables")

The training set contains  19622 Individuals  and  160 Variables
The testing set contains  20 Individuals  and  160 Variables

## Removing NA data

Here we remove all columns with missing values

In [5]:
trainData <- trainData[, colSums(is.na(trainData)) == 0] 
testData <- testData[, colSums(is.na(testData)) == 0] 

After removing the columns with missing data the training and testing dataset we obtain:

In [6]:
cat("The training set contains ", dim(trainData)[1], "Individuals", " and ",dim(trainData)[2], "Variables\n")
cat("The testing set contains ", dim(testData)[1], "Individuals", " and ",dim(testData)[2], "Variables")

The training set contains  19622 Individuals  and  93 Variables
The testing set contains  20 Individuals  and  60 Variables

Let's check if there are duplicate columns in our dataset

In [7]:
trainData <- trainData[!duplicated(lapply(trainData, summary))]
testData <- testData[!duplicated(lapply(testData, summary))]
cat("The training set contains ", dim(trainData)[1], "Individuals", " and ",dim(trainData)[2], "Variables\n")
cat("The testing set contains ", dim(testData)[1], "Individuals", " and ",dim(testData)[2], "Variables")

The training set contains  19622 Individuals  and  85 Variables
The testing set contains  20 Individuals  and  59 Variables

## Let's get rid of some columns that do not contribute much to the accelerometer measurements.

In [8]:
classe <- trainData$classe
trainRemove <- grepl("^X|timestamp|window", names(trainData))
trainData <- trainData[, !trainRemove]
trainCleaned <- trainData[, sapply(trainData, is.numeric)]
trainCleaned$classe <- classe
testRemove <- grepl("^X|timestamp|window", names(testData))
testData <- testData[, !testRemove]
testCleaned <- testData[, sapply(testData, is.numeric)]
cat("The training set contains ", dim(trainCleaned)[1], "Individuals", " and ",dim(trainCleaned)[2], "Variables\n")
cat("The testing set contains ", dim(testCleaned )[1], "Individuals", " and ",dim(testCleaned )[2], "Variables")

The training set contains  19622 Individuals  and  53 Variables
The testing set contains  20 Individuals  and  52 Variables

## Splitting the training dataset into training and validation data



In [9]:
set.seed(22519)
inTrain <- createDataPartition(trainCleaned$classe, p=0.70, list=F)
trainingData <- trainCleaned[inTrain,]
testingData <- trainingData[-inTrain, ]

## Model training and Prediction



Let’s start with a RF model. We will use the trainControl (http://www.inside- r.org/packages/cran/caret/docs/trainControl) function from caret. This function will cross-validate our data in order to find the best parameters for the RF model.

In [10]:
rf_model<-train(classe~.,data=trainingData,method="rf",trControl=trainControl(method="cv",number=5),prox=TRUE,allowParallel=TRUE)
print(rf_model)

Random Forest 

13737 samples
   52 predictor
    5 classes: 'A', 'B', 'C', 'D', 'E' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 10989, 10989, 10991, 10990, 10989 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
   2    0.9910460  0.9886726
  27    0.9914830  0.9892250
  52    0.9844941  0.9803815

Accuracy was used to select the optimal model using  the largest value.
The final value used for the model was mtry = 27. 


## Finally, we will predict the classess using our test dataset and evaluating the performance of our model

In [15]:
predRF <- predict(rf_model, testingData)

Now, we will bring the Consusion Matrix the the predicted values and the Classes

In [16]:
confusionMatrix(testingData$classe, predRF)

Confusion Matrix and Statistics

          Reference
Prediction    A    B    C    D    E
         A 1172    0    0    0    0
         B    0  783    0    0    0
         C    0    0  719    0    0
         D    0    0    0  692    0
         E    0    0    0    0  745

Overall Statistics
                                     
               Accuracy : 1          
                 95% CI : (0.9991, 1)
    No Information Rate : 0.2851     
    P-Value [Acc > NIR] : < 2.2e-16  
                                     
                  Kappa : 1          
 Mcnemar's Test P-Value : NA         

Statistics by Class:

                     Class: A Class: B Class: C Class: D Class: E
Sensitivity            1.0000   1.0000   1.0000   1.0000   1.0000
Specificity            1.0000   1.0000   1.0000   1.0000   1.0000
Pos Pred Value         1.0000   1.0000   1.0000   1.0000   1.0000
Neg Pred Value         1.0000   1.0000   1.0000   1.0000   1.0000
Prevalence             0.2851   0.1905   0.1749   0.16

### Finally we use the model to predict the class on the testing dataset.

In [13]:
final <-predict(rf_model, testingData[, -length(names(testingData))])


The accuracy of the model is:

In [17]:
acc <- postResample(predRF, testingData$classe)
acc

Accuracy    Kappa 
       1        1